# Модель рекомендации тариф для оператора мобильной связи  «Мегалайн».

**Цель проекта:** построить модель для задачи классификации, со значением accuracy не менее 0,75.

**План проекта:**
- Открыть файл с данными и изучите его.  
- Разделите исходные данные на обучающую, валидационную и тестовую выборки.
- Исследуйте качество разных моделей, меняя гиперпараметры.  
- Проверьте качество модели на тестовой выборке, достичь показателя accuracy не менее 0,75.
- Проверьте модели на адекватность.


**Описание данных**

**сalls** — количество звонков  
**minutes** — суммарная длительность звонков в минутах  
**messages** — количество sms-сообщений  
**mb_used** — израсходованный интернет-трафик в Мб  
**is_ultra** — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)  

## Откройте и изучите файл

Импортируем библиотеку pandas, а также библиотеки sklearn для  задач классификации.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 

In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')   
except BaseException:
    df = pd.read_csv('C:/Users/HP/Downloads/Модель для рекомендации тарифа оператора мобильной связи/users_behavior.csv')
      

Посмотрим на первые 5 строк датафрейма

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Посмотрим размерность первичного датафрейма

In [4]:
df_sh = df.shape
df_sh

(3214, 5)

Проверяем типы данных

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Проверяем пропуски

In [6]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Проверяем дубликаты; явные и неявыне

In [7]:
df.duplicated().sum()

0

In [8]:
df[df[['calls', 'minutes', 'messages', 'mb_used', 'is_ultra']].duplicated(keep=False)]

,calls,minutes,messages,mb_used,is_ultra


Данные представены верно, отстусвуют пропуски или дубликаты, типы данных присвоены адекватно.  
Теперь сосредоточимся на сути и закономерностях в самих данных.

**Разберемся чем отличаются пользователи с тарифом «Ультра» и «Смарт»**

In [9]:
df_0 = df.query('is_ultra == 0')
df_0.describe().round()

,calls,minutes,messages,mb_used,is_ultra
count,2229.0,2229.0,2229.0,2229.0,2229.0
mean,58.0,406.0,33.0,16208.0,0.0
std,26.0,185.0,28.0,5870.0,0.0
min,0.0,0.0,0.0,0.0,0.0
25%,40.0,274.0,10.0,12643.0,0.0
50%,60.0,411.0,28.0,16507.0,0.0
75%,76.0,530.0,51.0,20043.0,0.0
max,198.0,1390.0,143.0,38553.0,0.0


In [10]:
df_1 = df.query('is_ultra == 1')
df_1.describe().round()

,calls,minutes,messages,mb_used,is_ultra
count,985.0,985.0,985.0,985.0,985.0
mean,73.0,511.0,49.0,19469.0,1.0
std,44.0,308.0,48.0,10087.0,0.0
min,0.0,0.0,0.0,0.0,1.0
25%,41.0,276.0,6.0,11770.0,1.0
50%,74.0,503.0,38.0,19308.0,1.0
75%,104.0,730.0,79.0,26838.0,1.0
max,244.0,1632.0,224.0,49746.0,1.0


In [11]:
c = ((df_1['calls'].mean() / df_0['calls'].mean())-1).round(2)
mi = ((df_1['minutes'].mean() / df_0['minutes'].mean())-1).round(2)
me = ((df_1['messages'].mean() / df_0['messages'].mean())-1).round(2)
mb = ((df_1['mb_used'].mean() / df_0['mb_used'].mean())-1).round(2)
print('Соотношение между пользователями «Ультра» и «Смарт» по звонкам', c)
print('Соотношение между пользователями «Ультра» и «Смарт» по минутам разговора', mi)
print('Соотношение между пользователями «Ультра» и «Смарт» по сообщениям', me)
print('Соотношение между пользователями «Ультра» и «Смарт» по скаченным Мб ', mb)

Соотношение между пользователями «Ультра» и «Смарт» по звонкам 0.26
Соотношение между пользователями «Ультра» и «Смарт» по минутам разговора 0.26
Соотношение между пользователями «Ультра» и «Смарт» по сообщениям 0.48
Соотношение между пользователями «Ультра» и «Смарт» по скаченным Мб  0.2


In [12]:
ult = ((df_1['is_ultra'].count() / df['is_ultra'].count())).round(2)
print('Доля пользователей «Ультра» в датафрейме', ult)

Доля пользователей «Ультра» в датафрейме 0.31


**Вывод:** Исходя из полученных данных, видно что пользователи "Ульта" пользуются связью на 20%-48% больше чем пользователи "Смарт", это касается всех направлений: количество звонков, продолжительность звонков, количество сообщений и объема израсходованного интернет-трафика в Мб.
При этом пользователи "Ультра" составляют всего 31% от всего датафрейма.

## Разбейте данные на выборки

**Разделим датафреймы с обучающими признаками и целевым признаком на тренировочную выборку, валидационную и тестовую.**

In [13]:
df_train, dff = train_test_split(df, train_size=0.6, random_state=12345) 

In [14]:
df_test, df_valid = train_test_split(dff, train_size=0.5, random_state=12345) 

In [15]:
print(df_train.shape)
print(df_test.shape)
print(df_valid.shape)

(1928, 5)
(643, 5)
(643, 5)


In [16]:
(df_train.shape[0]+df_test.shape[0] + df_valid.shape[0])-df_sh[0]

0

При разделении данные не потерялись, можно идти дальше.

Проверим долю пользователей "Ультра" в новых выборках.

In [17]:
print('df_train:',(df_train.query('is_ultra == 1').count().max()/df_train.shape[0]))
print('df_test:',(df_test.query('is_ultra == 1').count().max()/df_test.shape[0]))
print('df_valid:',(df_valid.query('is_ultra == 1').count().max()/df_valid.shape[0]))

df_train: 0.30757261410788383
df_test: 0.2939346811819596
df_valid: 0.3157076205287714


Доля пользователей "Ультра" во всех новых датафреймакх очень близка к доле в изначальном датафреме -31%, значит обучение как и проверка будут потроены на адекватных данных.

**Разделим датафреймы на целевой признак и признаки для обучения.**

In [18]:
df_train_features = df_train.drop(['is_ultra'], axis=1)
df_train_target = df_train['is_ultra']

df_test_features = df_test.drop(['is_ultra'], axis=1)
df_test_target = df_test['is_ultra']

df_valid_features = df_valid.drop(['is_ultra'], axis=1)
df_valid_target = df_valid['is_ultra']

**Посмотрим на размерность новых датафреймов.**

In [19]:
print(df_train_features.shape)
print(df_train_target.shape)

print(df_test_features.shape)
print(df_test_target.shape)

print(df_valid_features.shape)
print(df_valid_target.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


Размерность польностью сохранена, можно переходить к моделированию.

## Исследуйте модели

Так как перед нами стоит задача классификации, а именно построить модель которая предложит подходящий тариф для пользователя «Ультра» или «Смарт», построим следующие модели: дерево решений, случайный лес, логистическая регрессия.

Модель - **Дерево решений**

In [20]:
best_model1 = None
best_result1 = 0
best_depth1 = 0
for depth in range(1, 11):
    model1 = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model1.fit(df_train_features, df_train_target)
    result1 = model1.score(df_valid_features, df_valid_target) 
    if result1 > best_result1:
            best_model1 = model1
            best_result1 = result1
            best_depth1 = depth
print("Accuracy наилучшей модели на валидационной выборке:", best_result1.round(2))
print("Количество ветвей модели на валидационной выборке:", best_depth1)  

Accuracy наилучшей модели на валидационной выборке: 0.8
Количество ветвей модели на валидационной выборке: 7


Модель - **Случайный лес**

In [21]:
best_model2 = None
best_result2 = 0
best_est2 = 0
best_depth2 = 0
for est in range(10, 51, 10):
    for depth in range(1, 11):
        model2 = RandomForestClassifier(random_state=12345, n_estimators= est, max_depth=depth) 
        model2.fit(df_train_features, df_train_target) 
        result2 = model2.score(df_valid_features, df_valid_target) 
        if result2 > best_result2:
            best_model2 = model2
            best_result2 = result2
            best_est2 = est
            best_depth2 = depth
print("Accuracy наилучшей модели на валидационной выборке:", best_result2.round(2))
print("Количество деревьев наилучшей модели на валидационной выборке:", best_est2)
print("Количество ветвей модели на валидационной выборке:", best_depth2)

Accuracy наилучшей модели на валидационной выборке: 0.81
Количество деревьев наилучшей модели на валидационной выборке: 10
Количество ветвей модели на валидационной выборке: 9


Модель - **Логистическая регрессия**

In [22]:
model3 = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model3.fit(df_train_features, df_train_target) 
result3 = model3.score(df_valid_features, df_valid_target) 
print("Accuracy модели Логистическая регрессия:", result3.round(2))

Accuracy модели Логистическая регрессия: 0.68


**Вывод:**  
Наилучшее качество показала модель `Случайный лес` Accuracy = 81%, при 10 деревьях с 9 ветвями. При этом модель обучалась дольше всего.  

`Дерево решений` на втором месте, тоже с высоким Accuracy = 80%, количество ветвей -7. Модель обучилась достаточно быстро.   

`Логистическая регрессия` показала наихудший результат Accuracy = 68%, обучение было самым быстрым.

## Проверка модели на тестовой выборке

In [24]:
model_b = RandomForestClassifier(random_state=12345, n_estimators= 10, max_depth=9) 
model_b.fit(df_test_features, df_test_target) 
result_b = model_b.score(df_test_features, df_test_target)
model_b_pred = model_b.predict(df_test_features) 
print('Accuracy модели Случайный лес на тестовой выборке, 10 деревьев, 9 ветвей:', result_b.round(2))

Accuracy модели Случайный лес на тестовой выборке, 10 деревьев, 9 ветвей: 0.89


**Вывод:**  
На тестовой выборке модель `Случайный лес` показала Accuracy -  89%.  

Модель `Случайный лес` показывает наилучшее качество, при том что задействовано всего 10 деревьев, что относительно немного, и не должно приводить к значительным временным издержкам.  
В связи с этим, для построения системы рекомендации для пользователей, которая будет предлагать переход на один из новых тарифов оператора мобильной связи «Мегалайн»: «Смарт» или «Ультра» - я предлгаю использовать модель `Случайный лес`.

## Проверка модели на адекватность

Посчитаем через DummyClassifier

In [26]:
from sklearn.dummy import DummyClassifier

In [27]:
dummy_model = DummyClassifier(random_state = 12345, strategy='most_frequent')
dummy_model.fit(df_train_features, df_train_target)
accuracy_score(dummy_model.predict(df_test_features), df_test_target)

0.7060653188180405

Посчитаем через среднее

In [28]:
1- df_test_target.mean()

0.7060653188180404

**Вывод:** 
Среднее по таргету  и по `DummyClassifier` тестовой выборки дает нам результат - 0,70.  
Accuracy модели `Случайный лес` на тестовой выборке (10 деревьев, 9 ветвей), дает результат - 0.89.  
Таким образом можно утверждать что обученная  модель `Случайный лес` предсказывает результат лучше чем  обычное усреднениие на 19%.